# Iterators, Generators and Classic Coroutines

Let's start with **Iterators**.

## Iterators

An **iterator** is a type of object that allow us to iterate through an **iterable**. 
- An example of an **iterable** is a `list`. 
- But really we can implement an **iterable** as just a protocol for iterators to work

Here, and just for showing purposes, let's create a footballer **iterable** for a fixed set of words. It will behave so that it only iterates over footballers with more than `x` ballon'd or. (Only doing for 2 or more for the example)

In [13]:
class BalonDeOrHolders:
    def __init__(self, how_many):
        self.how_many = how_many
        self.winners = {"Lionel Messi":"7",
"Cristiano Ronaldo":"5",
"Johan Cruyff":"3",
"Marco van Basten":"3",
"Michel Platini":"3",
"Franz Beckenbauer":"2",
"Ronaldo":"2",
"Alfredo Di Stefano":"2",
"Kevin Keegan":"2",
"Karl Heinz-Rummenigge":"2"}
    def __getitem__(self, index):
        filtered = [winner for winner in self.winners.items() if int(winner[1]) >= self.how_many]
        return filtered[index]
        

In [16]:
holders = BalonDeOrHolders(4)
for holder in holders:
    print(holder)

('Lionel Messi', '7')
('Cristiano Ronaldo', '5')


More commonly you would use the `__iter__` method:

In [27]:
class BalonDeOrHolders2:
    def __init__(self, how_many):
        self.how_many = how_many
        self.winners = {"Lionel Messi":"7",
"Cristiano Ronaldo":"5",
"Johan Cruyff":"3",
"Marco van Basten":"3",
"Michel Platini":"3",
"Franz Beckenbauer":"2",
"Ronaldo":"2",
"Alfredo Di Stefano":"2",
"Kevin Keegan":"2",
"Karl Heinz-Rummenigge":"2"}
    def __iter__(self):
        self.filtered = [winner for winner in self.winners.items() if int(winner[1]) >= self.how_many]
        self.index = 0
        return self
        
    def __next__(self):
        if self.index >= len(self.filtered):
            raise StopIteration()
        to_return = self.filtered[self.index]
        self.index += 1
        return to_return
        

In [28]:
holders = BalonDeOrHolders2(4)
for holder in holders:
    print(holder)

('Lionel Messi', '7')
('Cristiano Ronaldo', '5')


### NOTE:

It is bad practice to mix the **Iterable** with the **Iterator** in the same class as I did here. A dedicated **Iterator** class would be the way to go in this case

### An Iterator out of any function

We can also create an iterator out of any function we want

In [31]:
import random

def iterate_me_until_i_return_guess():
    return random.randint(1, 10)

iterator = iter(iterate_me_until_i_return_guess, 7)

In [32]:
for i in iterator:
    print(i)

5
6
4
5
8
10
9
10
4
5
4
6
2
6


You can obviously traverse iterators one step at a time:

In [36]:
iterator = iter(iterate_me_until_i_return_guess, 7)
print(next(iterator))
print(next(iterator))

9
2


## Generators

A **Generator** is a more idiomatic Python way to do what we did before, without the need to create an Iterator class
A generator is created by any function that has the **yield** keyword in its body. 
A function like that is called a Generator Function

In [37]:
class BalonDeOrHolders3:
    def __init__(self, how_many):
        self.how_many = how_many
        self.winners = {"Lionel Messi":"7",
"Cristiano Ronaldo":"5",
"Johan Cruyff":"3",
"Marco van Basten":"3",
"Michel Platini":"3",
"Franz Beckenbauer":"2",
"Ronaldo":"2",
"Alfredo Di Stefano":"2",
"Kevin Keegan":"2",
"Karl Heinz-Rummenigge":"2"}
        
    def holders(self):
        self.filtered = [winner for winner in self.winners.items() if int(winner[1]) >= self.how_many]
        for holder in self.filtered:
            yield holder

In [38]:
holders = BalonDeOrHolders3(3).holders()

for holder in holders:
    print(holder)

('Lionel Messi', '7')
('Cristiano Ronaldo', '5')
('Johan Cruyff', '3')
('Marco van Basten', '3')
('Michel Platini', '3')


In [39]:
holders = BalonDeOrHolders3(3).holders()

print(next(holders))
print(next(holders))

('Lionel Messi', '7')
('Cristiano Ronaldo', '5')


Really any function with a **yield** is a generator. no need to iterate as in the example above:

In [40]:
def yield_stuff():
    yield 'hola'
    yield 'ciao'
    
generator = yield_stuff()

print(generator)
print(next(generator))
print(next(generator))

<generator object yield_stuff at 0x1097b4cf0>
hola
ciao


Generators stop at every yield to give control back to the calling code

In [42]:
def try_in_different_places():
    print("looking locally")
    yield None
    print("looking online")
    yield "stuff"
    
generator = try_in_different_places()

print(next(generator))
print(next(generator))

looking locally
None
looking online
stuff


### NOTE

There are quite a few Generators available in the standard library mainly in the **itertools**, and **functools** modules

#### Subgenerators

A generator can delegate work to another generator using `yield from` from client side using the parent generator nothing changes it gets values yielded from both generators

In [45]:
def tweets(user):
    users = {'pedro': ['hola', 'ciao'],
     'mike': ['hi', 'bye']}
    for word in users[user]:
        yield word
        
def all_tweets():
    users = ['pedro', 'mike']
    for user in users:
        yield user
        print(f" --- tweets for {user}")
        yield from tweets(user)
        

for tweet in all_tweets():
    print(tweet)

pedro
 --- tweets for pedro
hola
ciao
mike
 --- tweets for mike
hi
bye


## Classic Coroutines

Classic coroutines are more or less generators used  a bit differently

In concrete the type signature of a coroutine is something like `Generator[YieldType, SendType, ReturnType]`

The key differences between native coroutines and generators are:

- Generators produce data for iteration

- Coroutines are in general consumers of data. Not at all related to iteration

In [57]:
def do_stuff_with_quality(quality):
    print(f"doing stuff 1 with quality {quality}")
def do_stuff_with_quality2(quality):
    print(f"doing stuff 2 with quality {quality}")
def do_stuff_with_quality3(quality):
    print(f"doing stuff 3 with quality {quality}")    
    
def step_by_step_coroutine():
    print("Started Coroutine")
    quality = yield
    do_stuff_with_quality(quality)
    quality = yield
    do_stuff_with_quality2(quality)
    quality = yield
    do_stuff_with_quality3(quality)   
    yield 

routine =  step_by_step_coroutine()

# Prime the coroutine
next(routine)

routine.send("max_quality")
routine.send("medium_quality")
routine.send("low_quality")
    

Started Coroutine
doing stuff 1 with quality max_quality
doing stuff 2 with quality medium_quality


You can see the main difference with the Generators we have seen is the use of `send` in the client to send data to the coroutine.

##### Coroutine States:

'GEN_CREATED'
Waiting to start execution.

'GEN_RUNNING'
Currently being executed by the interpreter.

'GEN_SUSPENDED'
Currently suspended at a yield expression.

'GEN_CLOSED'
Execution has completed.



Example in a while loop

It is common in many cases to have the `yield` inside a loop in the coroutine. you can imagine a code like the following:

In [59]:
import os
def buffer_to_file():
    f = open("/tmp/stuff.txt","w+")
    try:
        while True:
            contents = yield
            f.write(contents)
    except GeneratorExit:
            print("Closing File")
            f.close()
            

coroutine = buffer_to_file()
next(coroutine)

coroutine.send("hola como estas\n")
coroutine.send("yo estoy bien y tu\n")
coroutine.close()
        

Closing File


#### NOTE:

"Coroutines are a natural way of expressing many algorithms, such as simulations, games, asynchronous I/O, and other forms of event-driven programming or co-operative multitasking."

### Concurrency with a single thread

- Traditionally many concurrent tasks have been solved by using multithreading
- Coroutines offer a way to tackle certain types of concurrent tasks with a single thread
- Remember: Concurrency != Parallelism. Important to know that for achiving Parallelism a multithread (or multiprocess) approach is needed

In [63]:
def coroutine1():
    step = yield 0
    name = "co1"
    for n in range(100):
        print(f"In {name}. Step {step}")
        step = yield step

def coroutine2():
    step = yield 0
    name = "co2"
    for n in range(11):
        print(f"In {name}. Step {step}")
        step = yield step     
        

co1 = coroutine1()
co2 = coroutine2()

step1 = next(co1)
step2 = next(co2)

while True:
    if step1 < step2 * 10:
        step1 = co1.send(step1 + 1)
    else:
        step2 = co2.send(step2 + 1)
    

In co2. Step 1
In co1. Step 1
In co1. Step 2
In co1. Step 3
In co1. Step 4
In co1. Step 5
In co1. Step 6
In co1. Step 7
In co1. Step 8
In co1. Step 9
In co1. Step 10
In co2. Step 2
In co1. Step 11
In co1. Step 12
In co1. Step 13
In co1. Step 14
In co1. Step 15
In co1. Step 16
In co1. Step 17
In co1. Step 18
In co1. Step 19
In co1. Step 20
In co2. Step 3
In co1. Step 21
In co1. Step 22
In co1. Step 23
In co1. Step 24
In co1. Step 25
In co1. Step 26
In co1. Step 27
In co1. Step 28
In co1. Step 29
In co1. Step 30
In co2. Step 4
In co1. Step 31
In co1. Step 32
In co1. Step 33
In co1. Step 34
In co1. Step 35
In co1. Step 36
In co1. Step 37
In co1. Step 38
In co1. Step 39
In co1. Step 40
In co2. Step 5
In co1. Step 41
In co1. Step 42
In co1. Step 43
In co1. Step 44
In co1. Step 45
In co1. Step 46
In co1. Step 47
In co1. Step 48
In co1. Step 49
In co1. Step 50
In co2. Step 6
In co1. Step 51
In co1. Step 52
In co1. Step 53
In co1. Step 54
In co1. Step 55
In co1. Step 56
In co1. Step 57
In co1.

StopIteration: 

### Native Coroutines. (For next time)